<a href="https://colab.research.google.com/github/bsjin1122/deep-learning/blob/main/03_keras/tf_data_dataset.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# tf.data.Dataset
딥러닝 모델의 데이터셋 구축을 보다 편리하고 체계적이게 만들어주는 유틸 클래스이다.
- Dataset클래스가 제공해 주는 다양한 기능들을 활용하여 윈도우, 배치, 셔플 등의 다양한 기능들을 손쉽게 구성할 수 있으며 편리하다
- Dataset 클래스 메소드의 기능을 잘 이해하고 있지 않으면, 의도한대로 데이터셋 구성을 구현할 수 없어서 세부 기능과 역할에 대해 숙지하는 것이 바람직하다.

In [3]:
import numpy as np
import tensorflow as tf

# as_numpy_iterator
생성된 Dataset 클래스를 넘파이 배열로 반환한다.
- Dataset 클래스는 모든 요소가 텐서로 출력되지만, 넘파이 배열로 변환하면 요소 출력이 가능하다.

In [4]:
dataset = tf.data.Dataset.range(10)

list(dataset.as_numpy_iterator())

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9]

# apply
- 사용자가 정의한 변환 함수를 Dataset 클래스에 적용할 수 있다.

In [5]:
dataset = tf.data.Dataset.range(10)

def filter_five(x):
    return x.filter(lambda x: x < 5)

list(dataset.apply(filter_five).as_numpy_iterator())

[0, 1, 2, 3, 4]

# from_tensor_slices
list혹은 numpy 배열을 Dataset 클래스로 변환하는 메소드이다.
- Dataset 클래스의 유용한 기능을 사용하기 위해서는, Dataset 클래스로 변환해야 사용 가능하다.
from_tensor_sclices 로 변환 후에는 TensorSliceDataset으로 타입이 변환됨을 확인할 수가 있다.

In [6]:
ds = tf.data.Dataset.from_tensor_slices([1, 2, 3, 4, 5])
print(type(ds))
list(ds.as_numpy_iterator())

<class 'tensorflow.python.data.ops.from_tensor_slices_op._TensorSliceDataset'>


[1, 2, 3, 4, 5]

# iteration
반복문으로 요소들 출력 가능.

In [7]:
ds = tf.data.Dataset.from_tensor_slices([1, 2, 3, 4, 5])

for d in ds:
    print(d)

tf.Tensor(1, shape=(), dtype=int32)
tf.Tensor(2, shape=(), dtype=int32)
tf.Tensor(3, shape=(), dtype=int32)
tf.Tensor(4, shape=(), dtype=int32)
tf.Tensor(5, shape=(), dtype=int32)


# range

In [8]:
ds = tf.data.Dataset.range(1, 10, 2)

for d in ds:
    print(d)

tf.Tensor(1, shape=(), dtype=int64)
tf.Tensor(3, shape=(), dtype=int64)
tf.Tensor(5, shape=(), dtype=int64)
tf.Tensor(7, shape=(), dtype=int64)
tf.Tensor(9, shape=(), dtype=int64)


# batch
배치 크기를 batch 메소드에 지정하며 배치 크기만큼의 요소를 하나의 배치로 구성하여 반환
- 총 8개 요소 생성, 배치 크기 3개로 구성, -> 3개의 요소가 1개의 배치로 구성된 결과를 출력


In [9]:
ds = tf.data.Dataset.range(8)
for d in ds.batch(3):
    print(d)

tf.Tensor([0 1 2], shape=(3,), dtype=int64)
tf.Tensor([3 4 5], shape=(3,), dtype=int64)
tf.Tensor([6 7], shape=(2,), dtype=int64)


# drop_remainder
배치 구성 시 잔여 요소의 개수가 배치 크기보다 작다면 생성하지 않는다.
- 총 8개의 요소를 생성, 3개의 요소를 1개의 배치로 묶는데 3개의 요소를 2개의 배치로 묶고 난 후 잔여 요소의 개수가 2개인데 출력하지 않고 버려졌다.

In [10]:
ds = tf.data.Dataset.range(8)
for d in ds.batch(3, drop_remainder=True):
    print(d)

tf.Tensor([0 1 2], shape=(3,), dtype=int64)
tf.Tensor([3 4 5], shape=(3,), dtype=int64)


# window
- 시계열 데이터셋 구축 시 윈도우 구현
- 윈도우: 위니도우 크기만큼의 데이터를 하나로 묶어 생성하고자 할 때 사용된다.

In [11]:
ds = tf.data.Dataset.range(10)
ds = ds.window(5, shift=1, drop_remainder=False)
for d in ds:
    print(list(d.as_numpy_iterator()))

[0, 1, 2, 3, 4]
[1, 2, 3, 4, 5]
[2, 3, 4, 5, 6]
[3, 4, 5, 6, 7]
[4, 5, 6, 7, 8]
[5, 6, 7, 8, 9]
[6, 7, 8, 9]
[7, 8, 9]
[8, 9]
[9]


딥러닝 모델에 데이터셋 주입시 크기가 일정하게 맞춰있지 않으며 에러가 발생할 수 있으므로, drop_remainder = True로 설정하는 것이 일반적이다.

In [12]:
ds = tf.data.Dataset.range(10)
ds = ds.window(5, shift=1, drop_remainder=True)
for d in ds:
    print(list(d.as_numpy_iterator()))

[0, 1, 2, 3, 4]
[1, 2, 3, 4, 5]
[2, 3, 4, 5, 6]
[3, 4, 5, 6, 7]
[4, 5, 6, 7, 8]
[5, 6, 7, 8, 9]


In [13]:
ds = tf.data.Dataset.range(10)
ds = ds.window(5, shift=2, drop_remainder=True)
for d in ds:
    print(list(d.as_numpy_iterator())) #윈도우 구성 시 2칸씩 건너뛰어 생성

[0, 1, 2, 3, 4]
[2, 3, 4, 5, 6]
[4, 5, 6, 7, 8]


# flat_map
map 함수를 Dataset 클래스에 적용하여 flatten한 결과를 반환한다.
flatten한 결과를 반환하기 때문에, map함수를 적용한 결과가 2차원 배열이라 하더라도 결과는 1차원으로 flatten되어 반환한다.


In [14]:
ds = tf.data.Dataset.range(10)
ds = ds.window(5, shift=1, drop_remainder=True)
# ds = ds.flat_map(lambda w: w.batch(3))
for d in ds:
    print(d)

<_VariantDataset element_spec=TensorSpec(shape=(), dtype=tf.int64, name=None)>
<_VariantDataset element_spec=TensorSpec(shape=(), dtype=tf.int64, name=None)>
<_VariantDataset element_spec=TensorSpec(shape=(), dtype=tf.int64, name=None)>
<_VariantDataset element_spec=TensorSpec(shape=(), dtype=tf.int64, name=None)>
<_VariantDataset element_spec=TensorSpec(shape=(), dtype=tf.int64, name=None)>
<_VariantDataset element_spec=TensorSpec(shape=(), dtype=tf.int64, name=None)>


In [15]:
ds = tf.data.Dataset.range(10)
ds = ds.window(5, shift=1, drop_remainder=True)
ds = ds.flat_map(lambda w: w.batch(5))
for d in ds:
    print(d)

tf.Tensor([0 1 2 3 4], shape=(5,), dtype=int64)
tf.Tensor([1 2 3 4 5], shape=(5,), dtype=int64)
tf.Tensor([2 3 4 5 6], shape=(5,), dtype=int64)
tf.Tensor([3 4 5 6 7], shape=(5,), dtype=int64)
tf.Tensor([4 5 6 7 8], shape=(5,), dtype=int64)
tf.Tensor([5 6 7 8 9], shape=(5,), dtype=int64)


# shuffle
- 데이터셋을 무작위로 섞는다.

In [16]:
# shuffle을 해주지 않은 경우
ds = tf.data.Dataset.from_tensor_slices(np.arange(10))#.shuffle()

for d in ds:
    print(d)

tf.Tensor(0, shape=(), dtype=int64)
tf.Tensor(1, shape=(), dtype=int64)
tf.Tensor(2, shape=(), dtype=int64)
tf.Tensor(3, shape=(), dtype=int64)
tf.Tensor(4, shape=(), dtype=int64)
tf.Tensor(5, shape=(), dtype=int64)
tf.Tensor(6, shape=(), dtype=int64)
tf.Tensor(7, shape=(), dtype=int64)
tf.Tensor(8, shape=(), dtype=int64)
tf.Tensor(9, shape=(), dtype=int64)


In [17]:
# shuffle 설정
ds = tf.data.Dataset.from_tensor_slices(np.arange(10)).shuffle(buffer_size=5)
for d in ds:
    print(d)

tf.Tensor(1, shape=(), dtype=int64)
tf.Tensor(3, shape=(), dtype=int64)
tf.Tensor(0, shape=(), dtype=int64)
tf.Tensor(4, shape=(), dtype=int64)
tf.Tensor(7, shape=(), dtype=int64)
tf.Tensor(8, shape=(), dtype=int64)
tf.Tensor(2, shape=(), dtype=int64)
tf.Tensor(9, shape=(), dtype=int64)
tf.Tensor(6, shape=(), dtype=int64)
tf.Tensor(5, shape=(), dtype=int64)


# map

In [18]:
window_size=5

ds = tf.data.Dataset.range(10)
ds = ds.window(window_size, shift=1, drop_remainder=True)
ds = ds.flat_map(lambda w: w.batch(window_size))
ds = ds.shuffle(10)

# 첫 4개와 마지막 1개를 분리
ds = ds.map(lambda x: (x[:-2], x[-2:]))
for x, y in ds:
    print('train set: {}'.format(x))
    print('label set: {}'.format(y))
    print('==='*10)

train set: [0 1 2]
label set: [3 4]
train set: [4 5 6]
label set: [7 8]
train set: [2 3 4]
label set: [5 6]
train set: [3 4 5]
label set: [6 7]
train set: [1 2 3]
label set: [4 5]
train set: [5 6 7]
label set: [8 9]
